# Regular Expressions in Python

By far, the best course I have found on Regular Expressions is:  
https://www.safaribooksonline.com/videos/understanding-regular-expressions/9781491996300  

The best website I have found for learning and experimenting with Regular Expressions is:  
https://regex101.com/ 

## **re** module vs **regex** module

* **re** is part of standard library
* **regex** has latest PCRE with variable length look behind and more

In most cases, re and regex will do the same thing.  However with advanced regular expressions, you may find yourself wanting to use regex over re.

PCRE means Perl Compatible Regular Expressions

## Regular Expressions

In [1]:
import re

ml_text = """Machine learning is a method of data analysis that automates 
analytical model building. It is a branch of artificial intelligence 
based on the idea that systems can learn from data, identify patterns 
and make decisions with minimal human intervention.
"""

pattern = "minimal"

if re.search(pattern, ml_text):
    print('Match Found')
else:
    print('Match Not Found')

Match Found


In [2]:
# A Python pattern can be explictly compiled before usage
compiled_pattern = re.compile(pattern)
if compiled_pattern.search(ml_text):
    print('Match Found')
else:
    print('Match Not Found')

Match Found


### Note
If you do not explicitly compile the pattern, Python will do that for you.  It will also cache a few of the compiled patterns.

If you use a lot of patterns, it can be more time efficient to compile each pattern.

In [3]:
# Test for multiple patterns in a given string

# List of patterns to search for
patterns = ['learn', 'pattern']

for pattern in patterns:
    print(f'Searching for "{pattern}"')
    
    #Check for match
    if re.search(pattern, ml_text):
        print('Match was found. \n')
    else:
        print('No Match was found.\n')

Searching for "learn"
Match was found. 

Searching for "pattern"
Match was found. 



In [4]:
# using regex options in re.search()
pattern = "machine"
match = re.search(pattern, ml_text, flags=re.IGNORECASE)
if match:
    print('Match was found.')
    print(ml_text[match.start():match.end()])

Match was found.
Machine


In [5]:
# using regex options in pattern
match = re.search("(?i)machine", ml_text)
if match:
    print('Match was found.')
    print(ml_text[match.start():match.end()])

Match was found.
Machine


### re.search() Options vs RegEx Option

A regex option is also called a regex flag.

It is easier to read and maintain code using arguments to re.search() than to specify flags inside the regular expression.

On rare occasions, you may want regex flag to be in effect for only part of the pattern.  In this case the flag must be specified inside the regex.

In [6]:
# split examle
re.split('@', 'myname@email.com')

['myname', 'email.com']

In [7]:
# findall example
re.findall('is', ml_text, re.IGNORECASE)

['is', 'is', 'is', 'is']

In [8]:
def multi_re_find(patterns,phrase):
    """
    Takes in a list of regex patterns
    Prints a list of all matches
    """
    for pattern in patterns:
        print(f'{pattern!r}')
        print(re.findall(pattern,phrase))
        print('\n')

In [9]:
test_phrase = 'a.ab.aab..abb..aabbb...abbb'
test_patterns = [r'ab*',     # a followed by zero or more b's
                 r'ab+',     # a followed by one or more b's
                 r'ab?',     # a followed by zero or one b's
                 r'ab{3}',   # a followed by three b's
                 r'ab{2,3}', # a followed by two to three b's
                ]

multi_re_find(test_patterns,test_phrase)

'ab*'
['a', 'ab', 'a', 'ab', 'abb', 'a', 'abbb', 'abbb']


'ab+'
['ab', 'ab', 'abb', 'abbb', 'abbb']


'ab?'
['a', 'ab', 'a', 'ab', 'ab', 'a', 'ab', 'ab']


'ab{3}'
['abbb', 'abbb']


'ab{2,3}'
['abb', 'abbb', 'abbb']




In [10]:
test_phrase = 'a.ab.aab..abb..aabbb...abbb'

test_patterns = [r'[ab]',    # either a or b
                 r'a[ab]+']  # a followed by one or more a or b

multi_re_find(test_patterns,test_phrase)

'[ab]'
['a', 'a', 'b', 'a', 'a', 'b', 'a', 'b', 'b', 'a', 'a', 'b', 'b', 'b', 'a', 'b', 'b', 'b']


'a[ab]+'
['ab', 'aab', 'abb', 'aabbb', 'abbb']




In [11]:
test_phrase = 'This is a string! But it has punctuation. How can we remove it?'
re.findall('[^!.? ]+',test_phrase)

['This',
 'is',
 'a',
 'string',
 'But',
 'it',
 'has',
 'punctuation',
 'How',
 'can',
 'we',
 'remove',
 'it']

In [12]:
test_phrase = 'This is an example sentence. Lets see if we can find some letters.'

test_patterns=['[a-z]+',      # sequences of lower case letters
               '[A-Z]+',      # sequences of upper case letters
               '[a-zA-Z]+',   # sequences of lower or upper case letters
               '[A-Z][a-z]+'] # one upper case letter followed by lower case letters
                
multi_re_find(test_patterns,test_phrase)

'[a-z]+'
['his', 'is', 'an', 'example', 'sentence', 'ets', 'see', 'if', 'we', 'can', 'find', 'some', 'letters']


'[A-Z]+'
['T', 'L']


'[a-zA-Z]+'
['This', 'is', 'an', 'example', 'sentence', 'Lets', 'see', 'if', 'we', 'can', 'find', 'some', 'letters']


'[A-Z][a-z]+'
['This', 'Lets']




In [13]:
test_phrase = 'This is a string with some numbers 1233 and a symbol #hashtag'

test_patterns=[ r'\d+', # sequence of digits
                r'\D+', # sequence of non-digits
                r'\s+', # sequence of whitespace
                r'\S+', # sequence of non-whitespace
                r'\w+', # alphanumeric characters
                r'\W+', # non-alphanumeric
                ]

multi_re_find(test_patterns,test_phrase)

'\\d+'
['1233']


'\\D+'
['This is a string with some numbers ', ' and a symbol #hashtag']


'\\s+'
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']


'\\S+'
['This', 'is', 'a', 'string', 'with', 'some', 'numbers', '1233', 'and', 'a', 'symbol', '#hashtag']


'\\w+'
['This', 'is', 'a', 'string', 'with', 'some', 'numbers', '1233', 'and', 'a', 'symbol', 'hashtag']


'\\W+'
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' #']




In [14]:
s2 = r'\D'
print("%r" % s2)
print(f'{s2!r}')
print("%s" % s2)
print(f'{s2!s}')

'\\D'
'\\D'
\D
\D


## Experiments

In [15]:
s = '12abababcabx'
pattern = r'abc|abx'
re.findall(pattern, s)

['abc', 'abx']

In [16]:
s = 'An ant encoutnered an anteater'
pattern = r'anteater|antelope|ant'

In [17]:
# first match IS the match
match = re.search(pattern, s)
s[match.start():match.end()]

'ant'

In [18]:
# we can also look for all matches
re.findall(pattern, s)

['ant', 'anteater']

In [19]:
s = 'An anteater encoutnered an ant'
pattern = r'ant|anteater|antelope'

# first match IS the match
match = re.search(pattern, s)
s[match.start():match.end()]

'ant'

In [20]:
# we can also look for all matches
re.findall(pattern, s)

['ant', 'ant']

#### Note: usually faster to succeed with RegEx than fail

In [21]:
# whitespace says match whitespace
s = 'H2O'
pattern = r'H 2 O'
re.findall(pattern, s)

[]

In [22]:
s = 'H2O'
pattern = r'H 2 O'
re.findall(pattern, s, re.X)

['H2O']

In [23]:
s = 'H2O'
pattern = r'(?x)H 2 O'
re.findall(pattern, s)

['H2O']

In [24]:
#\Q literal \E does not work in Python
# but re.escape serves the same purpose
s = 'This is a ***$999 test'
pattern = re.escape(r'***$')
re.findall(pattern, s)

['***$']

In [25]:
s = 'This is some test string'
pattern = r'[aeiou]'
re.findall(pattern,s)

['i', 'i', 'o', 'e', 'e', 'i']

In [26]:
s = 'This is some test string'
pattern = r'[^aeiou]'
set(re.findall(pattern,s))

{' ', 'T', 'g', 'h', 'm', 'n', 'r', 's', 't'}

In [27]:
import regex
# this is regex, not re
s = 'This is some test string'
pattern = r'[[:lower:]]'
set(regex.findall(pattern,s))

{'e', 'g', 'h', 'i', 'm', 'n', 'o', 'r', 's', 't'}

In [28]:
s = 'This is some test string 1 2 3 456'
pattern = r'[^aeiou[:lower:]4-6]'
regex.findall(pattern,s)

['T', ' ', ' ', ' ', ' ', ' ', '1', ' ', '2', ' ', '3', ' ']

In [29]:
# Match everything up to first space character
s = 'This is some test string 1 2 3 456'
pattern = r'\S*'
match = re.search(pattern, s)
s[match.start():match.end()]

'This'

In [30]:
s = 'This is some test string 1 2 3 456'
pattern = r'.*\d'
match = re.search(pattern, s)
s[match.start():match.end()]

'This is some test string 1 2 3 456'

In [31]:
# not: not-word or digit
s = 'This is some test string 1 2 3 456'
pattern = r'[^\W\d]'
match = re.search(pattern, s)
s[match.start():match.end()]

'T'

In [32]:
print(regex.findall(pattern,s))

['T', 'h', 'i', 's', 'i', 's', 's', 'o', 'm', 'e', 't', 'e', 's', 't', 's', 't', 'r', 'i', 'n', 'g']


In [33]:
# . by default does not match \n
s = 'This is \na test.'
pattern = r'.*'
print(regex.findall(pattern,s))

['This is ', '', 'a test.', '']


In [34]:
# . by default does not match \n
s = 'This is \na test.'
pattern = r'.*'
print(regex.findall(pattern,s, regex.DOTALL))
print(re.findall(pattern,s, re.DOTALL))

['This is \na test.', '']
['This is \na test.', '']


In [35]:
# . by default does not match \n
s = 'This is not an isisis test.'
pattern = r'\s(is){1,3}\s'
print(regex.findall(pattern,s))

['is', 'is']


In [36]:
# . by default does not match \n
s = 'This is not an isisis test.'
pattern = r'\s((?:is){1,})\s'
match = regex.search(pattern,s)
print(match.groups())
print(regex.findall(pattern,s))

('is',)
['is', 'isisis']


In [37]:
# backtracking
# take as many alphanumeric characters as possible, ending in 2 digits
s = 'A123'
pattern = r'\w*\d\d'
print(regex.findall(pattern,s))

['A123']


In [38]:
# backtracking
# take as many alphanumeric characters as possible, ending in 2 digits
s = 'A1'
pattern = r'\w*\d\d'
print(regex.findall(pattern,s))

[]


In [39]:
# default is maximal match with minimal backtracking
s = '"Hello World!" and some "more" text "" and more'
pattern = r'".*"'
print(regex.findall(pattern,s))

['"Hello World!" and some "more" text ""']


In [40]:
# ? next to quantifier means minimal loop
pattern = r'".*?"'
print(regex.findall(pattern,s))

['"Hello World!"', '"more"', '""']


In [41]:
s = 'cp file1 file2'
pattern = r'(?:cp|mv|ln)\s+(\w+)\s+(\w+)'
print(regex.findall(pattern,s))

[('file1', 'file2')]


In [42]:
match = regex.search(pattern,s)
print(match.groups())
print(match.group(1)) # counting starts at 1!

('file1', 'file2')
file1


In [43]:
# with multiple matches, use findall
s = '< stuff inside angle brakets > this is not inside <more stuff>'
pattern = r'<(.*?)>'
groups = regex.findall(pattern,s)
groups

[' stuff inside angle brakets ', 'more stuff']

In [44]:
s = """
This is string1
This is string2
"""
pattern = r'\d$'
print(regex.findall(pattern,s))
print(regex.findall(pattern,s,regex.M))

['2']
['1', '2']


In [45]:
# negative (i.e. don't match) lookahead
s = 'end endwh1le endloop wh2le loop end'
pattern = r'end(?!\w)'
print(regex.findall(pattern,s))

['end', 'end']


In [46]:
# positive (i.e. match) lookahead
pattern = r'end(?=\s)'
print(regex.findall(pattern,s))

['end']


In [47]:
# positive (i.e. match) lookbehind
pattern = r'(?<!end)wh.le'
print(regex.findall(pattern,s))

['wh2le']


In [48]:
pattern = r'\bwh.le\b'
print(regex.findall(pattern,s))

['wh2le']


In [49]:
s = """
~123
~~1234
~~~12345
~~~~123456
"""
pattern = r'^~(?:~~)*\K\d+'
print(regex.findall(pattern,s,regex.M))

['123', '12345']


In [50]:
s = 'there are repeated repeated words in this string'
pattern = r'(\b\S+)\s+(\1)\b'
print(regex.findall(pattern,s))

[('repeated', 'repeated')]


In [51]:
s = """
~123
~~1234
~~~12345
~~~~123456
"""
pattern = r"""
(?x)       # turn on extended formatting
^~(?:~~)*  # start of string (in multiline mode) followed by ~, followed by ~~
\K\d+      # restart matching followed by 1 or more digits
"""
print(regex.findall(pattern,s,regex.M))

['123', '12345']


In [52]:
pattern_compiled = re.compile(r"""(?x)
(put|get)\s+
(files|buffers)\s+
in\s+
(\w+)
"""
                             )

In [53]:
s = 'put files in mydirectory'
print(pattern_compiled.findall(s))
match = pattern_compiled.search(s)
print(match.groups())

[('put', 'files', 'mydirectory')]
('put', 'files', 'mydirectory')


In [54]:
pattern_compiled = re.compile(r"""(?x)
(?P<CMD>put|get)\s+
(?P<TYPE>files|buffers)\s+
in\s+
(?P<TARGET>\w+)
"""
                             )

In [55]:
s = 'put files in mydirectory'
print(pattern_compiled.findall(s))
match = pattern_compiled.search(s)
print(match.groups())

[('put', 'files', 'mydirectory')]
('put', 'files', 'mydirectory')


In [56]:
match.groupdict()

{'CMD': 'put', 'TYPE': 'files', 'TARGET': 'mydirectory'}

In [57]:
print(match.group("CMD"))
print(match.group("TYPE"))
print(match.group("TARGET"))

put
files
mydirectory


In [58]:
pattern_compiled = re.compile(r'(dog)')
match = pattern_compiled.fullmatch("dog")
match.groups()

('dog',)

In [59]:
s = 'dog'
pattern = r'(dog)'
pc = re.compile(pattern)
match = pc.search(s)
print(match)
print(match.groups())

<re.Match object; span=(0, 3), match='dog'>
('dog',)


In [60]:
print(match[0]) # all groups
print(match[1]) # group 1

dog
dog


### Regex Module

In [61]:
import regex
regex.DEFAULT_VERSION == regex.V0

True

In [62]:
s = 'abcdefg'
pattern = r'[[a-z]--[aeiou]](?V0)'
regex.findall(pattern, s)

[]

In [63]:
s = 'abcdefg'
pattern = r'[[a-z]--[aeiou]](?V1)'
regex.findall(pattern, s)

['b', 'c', 'd', 'f', 'g']

In [64]:
pattern = r'[[a-z]--[aeiou]]'

In [65]:
print(re.findall(pattern, s))
print(regex.findall(pattern, s))
print(regex.findall(pattern, s, regex.V0))
print(regex.findall(pattern, s, regex.V1))

[]
[]
[]
['b', 'c', 'd', 'f', 'g']


/home/agni/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Possible nested set at position 1
  """Entry point for launching an IPython kernel.


In [66]:
s = 'a-beautiful-day'

In [67]:
import sys
print(sys.version)

# new in 3.7, this now the same as with regex.split() below
re.split('(?=-)', s)

3.7.1 (default, Dec 14 2018, 19:28:38) 
[GCC 7.3.0]


['a', '-beautiful', '-day']

In [68]:
regex.split('(?=-)', s, regex.V1)

['a', '-beautiful', '-day']

In [69]:
whos

Variable           Type        Data/Info
----------------------------------------
compiled_pattern   Pattern     re.compile('minimal')
groups             list        n=2
match              Match       <re.Match object; span=(0, 3), match='dog'>
ml_text            str         Machine learning is a met<...>mal human intervention.\n
multi_re_find      function    <function multi_re_find at 0x7f508c7201e0>
pattern            str         [[a-z]--[aeiou]]
pattern_compiled   Pattern     re.compile('(dog)')
patterns           list        n=2
pc                 Pattern     re.compile('(dog)')
re                 module      <module 're' from '/home/<...>/ml/lib/python3.7/re.py'>
regex              module      <module 'regex' from '/ho<...>/site-packages/regex.py'>
s                  str         a-beautiful-day
s2                 str         \D
sys                module      <module 'sys' (built-in)>
test_patterns      list        n=6
test_phrase        str         This is a string with som<...>2

In [70]:
regex.split('(?=-)', s)

['a', '-beautiful', '-day']

In [71]:
s = "<aa> b <ccde>"
r = r'<.*>'
mo = regex.match(r, s)
mo.captures()

['<aa> b <ccde>']

In [72]:
s = "<aa> b <ccde>"
r = r'<.*>'
mo = re.match(r, s)
mo.string

'<aa> b <ccde>'

In [73]:
s = "<aa> b <ccde>"
r = r'<.*?>'
mo = regex.match(r, s)
mo.captures()

['<aa>']

In [74]:
s = "<aa> b <ccde>"
r = r'<.*?>'
match_list = regex.findall(r, s)
match_list

['<aa>', '<ccde>']

In [75]:
s = "<aa> b <ccde>"
r = r'<.*>'
match_list = regex.findall(r, s)
match_list

['<aa> b <ccde>']

In [76]:
re.split(r'\W+', 'Words, words, and more words.')

['Words', 'words', 'and', 'more', 'words', '']

In [77]:
re.split(r'(?:\W+)', 'Words, words, and more words.')

['Words', 'words', 'and', 'more', 'words', '']

In [78]:
pattern = re.compile(r'(?P<digits>\d+) (?P<letters>[a-zA-Z])')
mo = pattern.search('123 45 6789 a234')
print(mo.group(0))
print(mo.group(1))
print(mo.group(2))
print(mo.groups())
print(mo.groupdict())

6789 a
6789
a
('6789', 'a')
{'digits': '6789', 'letters': 'a'}


In [79]:
pattern = re.compile(r'(?P<digits>\d+)')
mo = pattern.search('123 45 6789 a234')
print(mo.group(0))
print(mo.groups())
print(mo.groupdict())

123
('123',)
{'digits': '123'}


In [80]:
pattern = re.compile(r'(?P<digits>\d+)')
pattern.findall('123 45 6789 a234')

['123', '45', '6789', '234']

In [81]:
env = %env
env_list = [(key,value) for key,value in env.items()]
env_list.sort()
env_list[:5]

[('ADDR2LINE',
  '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-addr2line'),
 ('AR', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-ar'),
 ('AS', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-as'),
 ('CC', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-cc'),
 ('CFLAGS',
  '-march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe')]

In [82]:
print(type(env.keys()))

<class 'dict_keys'>


In [83]:
env = %env
keys = list(env.keys())
keys.sort()
for key in keys:
    m = regex.search(r'(?<=XDG)(\w+)', key)
    if m:
        print(m.group(1))

_CONFIG_DIRS
_CURRENT_DESKTOP
_DATA_DIRS
_MENU_PREFIX
_RUNTIME_DIR
_SEAT
_SESSION_DESKTOP
_SESSION_ID
_SESSION_TYPE
_VTNR


In [84]:
[k for k in keys if k.startswith('XDG')]

['XDG_CONFIG_DIRS',
 'XDG_CURRENT_DESKTOP',
 'XDG_DATA_DIRS',
 'XDG_MENU_PREFIX',
 'XDG_RUNTIME_DIR',
 'XDG_SEAT',
 'XDG_SESSION_DESKTOP',
 'XDG_SESSION_ID',
 'XDG_SESSION_TYPE',
 'XDG_VTNR']

In [85]:
[k for k in keys if regex.search(r'^XDG | ^PATH | ^LS_COLORS', k, flags=regex.X)]

['LS_COLORS',
 'PATH',
 'XDG_CONFIG_DIRS',
 'XDG_CURRENT_DESKTOP',
 'XDG_DATA_DIRS',
 'XDG_MENU_PREFIX',
 'XDG_RUNTIME_DIR',
 'XDG_SEAT',
 'XDG_SESSION_DESKTOP',
 'XDG_SESSION_ID',
 'XDG_SESSION_TYPE',
 'XDG_VTNR']

In [86]:
env = %env
env_list = [(key,value) for key,value in env.items() if not regex.search(r'^XDG | ^PATH | ^LS_COLORS | ^DBUS', key, flags=regex.X)]
env_list.sort()
env_list

[('ADDR2LINE',
  '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-addr2line'),
 ('AR', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-ar'),
 ('AS', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-as'),
 ('CC', '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-cc'),
 ('CFLAGS',
  '-march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe'),
 ('CLICOLOR', '1'),
 ('CLUTTER_IM_MODULE', 'xim'),
 ('COLORTERM', 'truecolor'),
 ('CONDA_BACKUP_F95',
  '/home/agni/anaconda3/envs/ml/bin/x86_64-conda_cos6-linux-gnu-f95'),
 ('CONDA_BACKUP_HOST', 'x86_64-conda_cos6-linux-gnu'),
 ('CONDA_DEFAULT_ENV', 'ml'),
 ('CONDA_EXE', '/home/agni/anaconda3/bin/conda'),
 ('CONDA_PREFIX', '/home/agni/anaconda3/envs/ml'),
 ('CONDA_PREFIX_1', '/home/agni/anaconda3'),
 ('CONDA_PROMPT_MODIFIER', '(ml) '),
 ('CONDA_PYTHON_EXE', '/home/agni/anaconda3/bin/python'),
 ('CONDA_SHLVL', '2'),
 ('CPP', '/home/agni/anaconda3/envs/ml/bin/x

In [87]:
# lsmagic
# pdoc regex

In [88]:
whos

Variable           Type        Data/Info
----------------------------------------
compiled_pattern   Pattern     re.compile('minimal')
env                dict        n=118
env_list           list        n=105
groups             list        n=2
key                str         _CONDA_PYTHON_SYSCONFIGDATA_NAME
keys               list        n=118
m                  NoneType    None
match              Match       <re.Match object; span=(0, 3), match='dog'>
match_list         list        n=1
ml_text            str         Machine learning is a met<...>mal human intervention.\n
mo                 Match       <re.Match object; span=(0, 3), match='123'>
multi_re_find      function    <function multi_re_find at 0x7f508c7201e0>
pattern            Pattern     re.compile('(?P<digits>\\d+)')
pattern_compiled   Pattern     re.compile('(dog)')
patterns           list        n=2
pc                 Pattern     re.compile('(dog)')
r                  str         <.*>
re                 module      <module

In [89]:
%psearch env*

env
env_list

In [90]:
timeit pass

5.89 ns ± 0.0212 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [91]:
who

compiled_pattern	 env	 env_list	 groups	 key	 keys	 m	 match	 match_list	 
ml_text	 mo	 multi_re_find	 pattern	 pattern_compiled	 patterns	 pc	 r	 re	 
regex	 s	 s2	 sys	 test_patterns	 test_phrase	 


In [92]:
whos

Variable           Type        Data/Info
----------------------------------------
compiled_pattern   Pattern     re.compile('minimal')
env                dict        n=118
env_list           list        n=105
groups             list        n=2
key                str         _CONDA_PYTHON_SYSCONFIGDATA_NAME
keys               list        n=118
m                  NoneType    None
match              Match       <re.Match object; span=(0, 3), match='dog'>
match_list         list        n=1
ml_text            str         Machine learning is a met<...>mal human intervention.\n
mo                 Match       <re.Match object; span=(0, 3), match='123'>
multi_re_find      function    <function multi_re_find at 0x7f508c7201e0>
pattern            Pattern     re.compile('(?P<digits>\\d+)')
pattern_compiled   Pattern     re.compile('(dog)')
patterns           list        n=2
pc                 Pattern     re.compile('(dog)')
r                  str         <.*>
re                 module      <module

In [93]:
who_ls

['compiled_pattern',
 'env',
 'env_list',
 'groups',
 'key',
 'keys',
 'm',
 'match',
 'match_list',
 'ml_text',
 'mo',
 'multi_re_find',
 'pattern',
 'pattern_compiled',
 'patterns',
 'pc',
 'r',
 're',
 'regex',
 's',
 's2',
 'sys',
 'test_patterns',
 'test_phrase']